In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print("Output Dir:")
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# main.py - Combined ECAPA-TDNN code (Python 3.10)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from typing import List, Tuple

# ========================
# dataLoader.py contents
# ========================
class CustomDataset(data.Dataset):
    def __init__(self, data_list: List[torch.Tensor], labels: torch.Tensor):
        self.data_list = data_list
        self.labels = labels

    def __len__(self) -> int:
        return len(self.data_list)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.data_list[idx], self.labels[idx]

# ========================
# model.py contents
# ========================
class ECAPA_TDNN(nn.Module):
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.fc(x)

# ========================
# loss.py contents
# ========================
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes: int, embedding_dim: int):
        super().__init__()
        self.fc = nn.Linear(embedding_dim, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, embeddings: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        logits = self.fc(embeddings)
        return self.loss_fn(logits, labels)

# ========================
# tools.py contents
# ========================
def compute_accuracy(predictions: torch.Tensor, labels: torch.Tensor) -> float:
    _, predicted = torch.max(predictions, 1)
    return (predicted == labels).sum().item() / labels.size(0)

# ========================
# ECAPAModel.py & trainECAPAModel.py contents
# ========================
def train_model(model: nn.Module, train_loader: data.DataLoader, criterion: nn.Module, optimizer: optim.Optimizer, epochs: int = 10) -> None:
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Main execution
if __name__ == "__main__":
    input_dim, output_dim, num_classes = 256, 512, 10
    train_data = torch.randn(100, input_dim)
    train_labels = torch.randint(0, num_classes, (100,))
    train_dataset = CustomDataset(train_data.tolist(), train_labels)
    train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    model = ECAPA_TDNN(input_dim, output_dim)
    criterion = SoftmaxLoss(num_classes, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_model(model, train_loader, criterion, optimizer)

In [ ]:
print("Done")